In [29]:
# Python
import pandas as pd
import numpy as np
#
from pathlib import Path
from datetime import datetime
#
import xgboost
import utilities.xgboost_utils as xgboost_utils
#
import importlib
import utilities.train_test.train_test as train_test
import utilities.mpt_utils as mpt_utils
import utilities.variables as variables
import utilities.plots as plots

## Data

In [30]:
df = pd.read_csv('../../../data/df_monthly_prices_complete_euro.csv', index_col='Date')
df_pct = pd.read_csv('../../../data/df_monthly_returns_complete.csv', index_col='Date')
df_overview = pd.read_csv('../../../data/df_overview.csv', index_col=0)

In [31]:
file = Path("../../../data/df_tabular.csv")

if file.exists():
    df_tabular = pd.read_csv(file, index_col=0)
else:
    importlib.reload(train_test)
    df_tabular = train_test.get_dataframe_tabular(df_pct)
    df_tabular.to_csv(file)

## Direct forecasting

## XGBoost Univariate - 1 Month

In [33]:
importlib.reload(train_test)
importlib.reload(xgboost_utils)
importlib.reload(variables)
#
df_tab_filtered = df_tabular.tail(variables.TRAIN_MONTH_NR*len(df.columns))
X_train, y_train, X_test, y_test, min_datestr = train_test.split_train_test_tabular(df_tab_filtered, months=variables.TEST_MONTHS_NR)
#
model_1m, score_1m = xgboost_utils.get_xgboost_trained_model(X_train, y_train, X_test, y_test)

[0]	validation_0-rmse:5.54953	validation_1-rmse:0.52412
[1]	validation_0-rmse:4.71811	validation_1-rmse:0.80544
[2]	validation_0-rmse:4.01113	validation_1-rmse:1.07080
[3]	validation_0-rmse:3.41032	validation_1-rmse:1.30500
[4]	validation_0-rmse:2.90033	validation_1-rmse:1.50332
[5]	validation_0-rmse:2.46686	validation_1-rmse:1.67303
[6]	validation_0-rmse:2.09857	validation_1-rmse:1.81942
[7]	validation_0-rmse:1.78608	validation_1-rmse:1.94295
[8]	validation_0-rmse:1.52087	validation_1-rmse:2.04825
[9]	validation_0-rmse:1.29557	validation_1-rmse:2.13896
[10]	validation_0-rmse:1.10527	validation_1-rmse:2.21517
[11]	validation_0-rmse:0.94426	validation_1-rmse:2.28004
[12]	validation_0-rmse:0.80795	validation_1-rmse:2.33523
[13]	validation_0-rmse:0.69265	validation_1-rmse:2.38214
[14]	validation_0-rmse:0.59576	validation_1-rmse:2.42205
[15]	validation_0-rmse:0.51467	validation_1-rmse:2.45596
[16]	validation_0-rmse:0.44700	validation_1-rmse:2.48480
[17]	validation_0-rmse:0.39092	validation

In [34]:
importlib.reload(train_test)
months_1m = 1
X_train_1m, y_train_1m, X_test_1m, y_test_1m, min_datestr = train_test.split_train_test_tabular(df_tabular, months=1)
# reset index
X_train_1m.reset_index(drop=True, inplace=True)
y_train_1m.reset_index(drop=True, inplace=True)
X_test_1m.reset_index(drop=True, inplace=True)
y_test_1m.reset_index(drop=True, inplace=True)

In [35]:
dt = datetime.strptime(min_datestr, '%Y-%m-%d')

# get all training months without the last month
X_train_input_1m = X_train_1m.head(len(X_train_1m) - len(df_pct.columns) * months_1m)
# Get last month of the training dataset and use as input to predict the next month
X_test_input_1m = X_train_1m.tail(len(df_pct.columns) * months_1m)

# Trained 
y_train_pred_1m = model_1m.predict(X_train_input_1m)
# Predictions
y_test_pred_1m = model_1m.predict(X_test_input_1m)

### 1 month Actual vs Prediction

In [36]:
importlib.reload(mpt_utils)
#
y_train_mean_pred_1m, y_test_mean_pred_1m = mpt_utils.get_train_test_mean_pred(y_train_pred_1m, y_test_pred_1m, len(df_pct.columns))
#
mpt_utils.generate_plot(df_pct, df_tabular, y_train_mean_pred_1m, y_test_mean_pred_1m)

#### Allocation - 1 Month

In [37]:
importlib.reload(mpt_utils)

# Get train true values followed with predicted month/s
y_train_1m_list = y_train_1m['m_return_target(t+1)'].tolist()
y_test_1m_list = y_test_pred_1m.tolist()
#
df_to_evaluate_1m = mpt_utils.get_df_from_pred_list(df_pct, y_train_1m_list, y_test_1m_list)
df_to_evaluate_1m = df_to_evaluate_1m - 1
df_to_evaluate_1m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.000000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.000000,-0.060000,...,0.00000,0.000000,0.000000,0.070000,0.000000,-0.340000,0.000000,0.000000,0.000000,-0.010000
1,0.000000,0.000000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.040000,0.050000,...,0.00000,0.000000,0.000000,0.090000,0.000000,-0.160000,0.000000,0.000000,0.000000,0.090000
2,0.010000,0.000000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.020000,-0.060000,...,0.00000,0.290000,-0.090000,-0.020000,-0.040000,0.690000,-0.140000,-0.240000,0.000000,0.050000
3,0.000000,-0.030000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.100000,-0.080000,...,-0.48000,0.000000,0.000000,0.010000,0.000000,-0.260000,0.000000,0.000000,0.000000,0.040000
4,0.000000,0.000000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.000000,0.020000,...,1.18000,0.000000,0.000000,0.050000,0.000000,0.150000,0.000000,0.000000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-0.050000,0.100000,0.060000,0.020000,0.140000,0.120000,0.120000,0.060000,0.060000,0.060000,...,0.09000,0.070000,0.160000,0.050000,-0.010000,0.080000,0.030000,-0.020000,-0.140000,-0.020000
283,0.000000,-0.040000,-0.040000,-0.030000,-0.050000,-0.040000,-0.060000,-0.060000,-0.050000,0.110000,...,0.07000,-0.090000,0.180000,-0.110000,0.010000,-0.100000,0.190000,0.010000,0.140000,-0.020000
284,0.190000,0.080000,0.020000,0.060000,0.120000,0.220000,0.050000,0.090000,0.070000,-0.080000,...,0.10000,0.070000,0.060000,0.160000,-0.060000,-0.040000,0.140000,0.280000,-0.100000,0.090000
285,-0.040000,-0.220000,0.060000,0.080000,-0.020000,-0.020000,0.030000,0.070000,0.100000,0.060000,...,-0.05000,0.040000,0.160000,-0.130000,-0.040000,0.000000,-0.030000,-0.070000,-0.110000,0.040000


In [38]:
importlib.reload(mpt_utils)

weights_1m, mu_1m, S_1m, weights_all_1m = mpt_utils.portfolio_and_plot(df_to_evaluate_1m, df)

Expected annual return: 34.6%
Annual volatility: 35.0%
Sharpe Ratio: 0.93
-- Allocation --
{'NVDA': 4, 'PLUS.L': 1, '7552.T': 13, '8129.T': 9, 'LRN': 5, '2760.T': 10, '7944.T': 11, '6417.T': 18, '7451.T': 6, '7433.T': 6, '9433.T': 6, 'SFM': 2, '7984.T': 11, '9101.T': 5, '9697.T': 10, 'PGR': 1, 'FCN': 1, '8114.T': 5, '7868.T': 47, 'PECO': 4, '3191.T': 11, '3659.T': 7, 'AGYS': 1, '7974.T': 2, '1911.T': 2, 'CPRX': 7, '1973.T': 5, '7734.T': 3, '8022.T': 1, '9107.T': 6, '9413.T': 4, '8060.T': 3, '3050.T': 10, '2767.T': 6, '6430.T': 3, '6460.T': 4, '9404.T': 4, 'FINV': 15, '8283.T': 2, '7906.T': 5, 'BIRK': 1, 'BBW': 1, 'OMI': 3, '8012.T': 2, '4812.T': 1, '2670.T': 2, '9434.T': 3, '2733.T': 2, 'PMTS': 2, '9882.T': 2, '7994.T': 2, '8309.T': 1, '7599.T': 1}
-- Weights Percentage --
{'7984.T': 0.0248, 'NVDA': 0.0649, '7944.T': 0.0344, '7433.T': 0.0291, '2760.T': 0.0358, '7552.T': 0.046, '6417.T': 0.033, 'LRN': 0.0391, '9101.T': 0.0244, '8129.T': 0.0427, 'SFM': 0.0252, '8114.T': 0.0202, '7451.T':

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Overview - 1 Month

In [39]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(weights_all_1m, mu_1m, S_1m, df_pct)

,Share Count,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
NVDA,4,0.081346,18.08%,-3.33%,209.80%
PLUS.L,1,-0.015769,13.88%,-32.76%,131.69%
7552.T,13,0.028269,28.26%,-17.65%,18.02%
8129.T,9,-0.017500,29.91%,-5.74%,-9.19%
LRN,5,-0.050577,19.37%,-8.93%,-0.92%
2760.T,10,0.041731,35.66%,9.08%,61.83%
7944.T,11,-0.285385,30.78%,-467.40%,876.44%
6417.T,18,0.035385,34.34%,-8.82%,-23.01%
7451.T,6,0.020000,94.12%,-3.79%,4.61%
7433.T,6,0.086346,1.65%,4.96%,-1.26%


## XGBoost Univariate - 6 Months

In [40]:
importlib.reload(train_test)
importlib.reload(xgboost_utils)
#
df_tab_filtered = df_tabular.tail(variables.TRAIN_MONTH_NR*len(df.columns))
X_train, y_train, X_test, y_test, min_datestr = train_test.split_train_test_tabular(df_tab_filtered, months=variables.TEST_MONTHS_NR)
#
model_6m, score_6m = xgboost_utils.get_xgboost_trained_model(X_train, y_train, X_test, y_test)

[0]	validation_0-rmse:5.54953	validation_1-rmse:0.52412
[1]	validation_0-rmse:4.71811	validation_1-rmse:0.80544
[2]	validation_0-rmse:4.01113	validation_1-rmse:1.07080
[3]	validation_0-rmse:3.41032	validation_1-rmse:1.30500
[4]	validation_0-rmse:2.90033	validation_1-rmse:1.50332
[5]	validation_0-rmse:2.46686	validation_1-rmse:1.67303
[6]	validation_0-rmse:2.09857	validation_1-rmse:1.81942
[7]	validation_0-rmse:1.78608	validation_1-rmse:1.94295
[8]	validation_0-rmse:1.52087	validation_1-rmse:2.04825
[9]	validation_0-rmse:1.29557	validation_1-rmse:2.13896
[10]	validation_0-rmse:1.10527	validation_1-rmse:2.21517
[11]	validation_0-rmse:0.94426	validation_1-rmse:2.28004
[12]	validation_0-rmse:0.80795	validation_1-rmse:2.33523
[13]	validation_0-rmse:0.69265	validation_1-rmse:2.38214
[14]	validation_0-rmse:0.59576	validation_1-rmse:2.42205
[15]	validation_0-rmse:0.51467	validation_1-rmse:2.45596
[16]	validation_0-rmse:0.44700	validation_1-rmse:2.48480
[17]	validation_0-rmse:0.39092	validation

In [41]:
months_6m = 6
X_train_6m, y_train_6m, X_test_6m, y_test_6m, min_datestr = train_test.split_train_test_tabular(df_tabular, months=6,
                                                                                                target_key='m_return_target(t+6)')

In [42]:
dt = datetime.strptime(min_datestr, '%Y-%m-%d')

# get all training months without the last 6 months
X_train_input_6m = X_train_6m.head(len(X_train_6m) - len(df_pct.columns)*months_6m)
# Get last 6 months of the training dataset and use as input to predict the next 6 months
X_test_input_6m = X_train_6m.tail(len(df_pct.columns)*months_6m)

# Trained 
y_train_pred_6m = model_6m.predict(X_train_input_6m)
# Predictions
y_test_pred_6m = model_6m.predict(X_test_input_6m)

### 6 Months Actual vs Prediction

In [43]:
importlib.reload(mpt_utils)
#
y_train_mean_pred_6m, y_test_mean_pred_6m = mpt_utils.get_train_test_mean_pred(y_train_pred_6m, y_test_pred_6m, len(df_pct.columns))
#
plots.generate_timeseries_plot(df_pct, df_tabular, y_train_mean_pred_6m, y_test_mean_pred_6m)

#### Allocation

In [44]:
# Get train true values followed with predicted month/s
y_train_6m_list = y_train_6m['m_return_target(t+6)'].tolist()
y_pred_6m_list = y_test_pred_6m.tolist()
#
df_to_evaluate_6m = mpt_utils.get_df_from_pred_list(df_pct, y_train_1m_list, y_pred_6m_list)
df_to_evaluate_6m = df_to_evaluate_6m - 1
df_to_evaluate_6m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.000000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.000000,-0.060000,...,0.000000,0.000000,0.000000,0.070000,0.000000,-0.340000,0.000000,0.000000,0.000000,-0.010000
1,0.000000,0.000000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.040000,0.050000,...,0.000000,0.000000,0.000000,0.090000,0.000000,-0.160000,0.000000,0.000000,0.000000,0.090000
2,0.010000,0.000000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.020000,-0.060000,...,0.000000,0.290000,-0.090000,-0.020000,-0.040000,0.690000,-0.140000,-0.240000,0.000000,0.050000
3,0.000000,-0.030000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.100000,-0.080000,...,-0.480000,0.000000,0.000000,0.010000,0.000000,-0.260000,0.000000,0.000000,0.000000,0.040000
4,0.000000,0.000000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.000000,0.020000,...,1.180000,0.000000,0.000000,0.050000,0.000000,0.150000,0.000000,0.000000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0.002740,0.026169,-0.008425,-0.074686,0.051148,-0.018812,-0.006314,-0.152143,-0.005696,-0.025223,...,-0.052690,-0.003058,0.007544,0.005687,0.018953,-0.016136,0.024807,-0.001017,-0.003167,-0.020670
288,0.017626,-0.003381,0.015280,0.120744,-0.020245,0.099891,0.000865,0.177171,0.058965,-0.028522,...,0.047689,-0.042267,0.011395,-0.095516,-0.017072,-0.048118,0.017243,-0.066740,0.011619,-0.062361
289,-0.022145,0.035312,0.028185,0.059979,-0.087291,-0.070529,-0.209806,0.251381,-0.061305,-0.019806,...,0.017754,0.041611,-0.005537,0.057154,0.026266,0.069425,0.012728,0.092886,0.001025,0.053037
290,0.058999,0.034803,0.015719,-0.011530,-0.020268,0.048157,0.028170,0.035308,-0.014318,-0.018847,...,-0.018098,0.018173,-0.038693,0.035211,-0.003429,0.010325,-0.010251,0.020092,-0.004218,0.016839


In [45]:
importlib.reload(mpt_utils)

weights_6m, mu_6m, S_6m, weights_all_6m = mpt_utils.portfolio_and_plot(df_to_evaluate_6m, df)

Expected annual return: 34.2%
Annual volatility: 30.5%
Sharpe Ratio: 1.05
-- Allocation --
{'PLUS.L': 1, '7552.T': 10, '8129.T': 7, 'LRN': 3, '3191.T': 14, '7906.T': 10, '7451.T': 6, '6146.T': 1, 'PGR': 1, '7944.T': 34, '6417.T': 15, '2760.T': 8, '7433.T': 5, 'PMTS': 2, '3608.T': 12, '9697.T': 8, 'SFM': 1, 'NVDA': 2, '8283.T': 4, '2767.T': 8, '7974.T': 2, '4218.T': 3, 'PECO': 4, '9101.T': 3, '7984.T': 8, '7936.T': 3, '8022.T': 1, '9987.T': 2, '8173.T': 5, '8060.T': 3, '3050.T': 10, '3132.T': 1, '7734.T': 3, '8114.T': 3, '3765.T': 3, '6745.T': 4, 'API.L': 1, 'OMI': 6, '6430.T': 3, '9433.T': 3, '9107.T': 4, '2784.T': 4, '2670.T': 3, 'CPRX': 3, '7994.T': 4, 'GEN': 2, '2685.T': 1, '8012.T': 2, '9882.T': 3, '7868.T': 20, '9434.T': 3, '8923.T': 2, '7867.T': 1, '9413.T': 1, '7762.T': 4, '9869.T': 1, '6460.T': 2, '7239.T': 1, '7459.T': 1, '9069.T': 4, 'BBW': 1, '6718.T': 1, '8601.T': 1}
-- Weights Percentage --
{'7906.T': 0.0306, 'NVDA': 0.0213, '7944.T': 0.0263, '7433.T': 0.0245, '8283.T': 0.

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Overview table - 6 Months

In [46]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(weights_all_6m, mu_6m, S_6m, df_pct)

,Share Count,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
PLUS.L,1,0.018387,22.10%,-32.76%,131.69%
7552.T,10,0.013387,13.59%,-17.65%,18.02%
8129.T,7,-0.002419,33.77%,-5.74%,-9.19%
LRN,3,-0.060484,26.66%,-8.93%,-0.92%
3191.T,14,-0.008387,-3.92%,-12.71%,26.92%
...,...,...,...,...,...
7459.T,1,0.032903,114.29%,4.50%,-10.67%
9069.T,4,0.035645,35.48%,2.27%,20.42%
BBW,1,0.146452,32.42%,-59.69%,-20.85%
6718.T,1,0.080645,11.50%,30.14%,-1.44%


## XGBoost Univariate - 12 Months 

In [47]:
importlib.reload(train_test)
importlib.reload(xgboost_utils)
#
df_tab_filtered = df_tabular.tail(variables.TRAIN_MONTH_NR*len(df.columns))
X_train, y_train, X_test, y_test, min_datestr = train_test.split_train_test_tabular(df_tab_filtered, months=variables.TEST_MONTHS_NR)
#
model_12m, score_12m = xgboost_utils.get_xgboost_trained_model(X_train, y_train, X_test, y_test)

[0]	validation_0-rmse:5.54953	validation_1-rmse:0.52412
[1]	validation_0-rmse:4.71811	validation_1-rmse:0.80544
[2]	validation_0-rmse:4.01113	validation_1-rmse:1.07080
[3]	validation_0-rmse:3.41032	validation_1-rmse:1.30500
[4]	validation_0-rmse:2.90033	validation_1-rmse:1.50332
[5]	validation_0-rmse:2.46686	validation_1-rmse:1.67303
[6]	validation_0-rmse:2.09857	validation_1-rmse:1.81942
[7]	validation_0-rmse:1.78608	validation_1-rmse:1.94295
[8]	validation_0-rmse:1.52087	validation_1-rmse:2.04825
[9]	validation_0-rmse:1.29557	validation_1-rmse:2.13896
[10]	validation_0-rmse:1.10527	validation_1-rmse:2.21517
[11]	validation_0-rmse:0.94426	validation_1-rmse:2.28004
[12]	validation_0-rmse:0.80795	validation_1-rmse:2.33523
[13]	validation_0-rmse:0.69265	validation_1-rmse:2.38214
[14]	validation_0-rmse:0.59576	validation_1-rmse:2.42205
[15]	validation_0-rmse:0.51467	validation_1-rmse:2.45596
[16]	validation_0-rmse:0.44700	validation_1-rmse:2.48480
[17]	validation_0-rmse:0.39092	validation

In [48]:
months_12m = 12
X_train_12m, y_train_12m, X_test_12m, y_test_12m, min_datestr = train_test.split_train_test_tabular(df_tabular, months=months_12m,
                                                                                                    target_key='m_return_target(t+12)')

In [49]:
dt = datetime.strptime(min_datestr, '%Y-%m-%d')

# get all training months without the last 12 months
X_train_input_12m = X_train_12m.head(len(X_train_12m) - len(df_pct.columns)*months_12m)
# Get last 6 months of the training dataset and use as input to predict the next 12 months
X_test_input_12m = X_train_12m.tail(len(df_pct.columns)*months_12m)

# Trained 
y_train_pred_12m = model_12m.predict(X_train_input_12m)
# Predictions
y_test_pred_12m = model_6m.predict(X_test_input_12m)

### 12 Months Actual vs Prediction

In [50]:
importlib.reload(mpt_utils)
#
y_train_mean_pred_12m, y_test_mean_pred_12m = mpt_utils.get_train_test_mean_pred(y_train_pred_12m, y_test_pred_12m, len(df_pct.columns))
#
mpt_utils.generate_plot(df_pct, df_tabular, y_train_mean_pred_12m, y_test_mean_pred_12m)

#### Allocation - 12 Months

In [51]:
importlib.reload(mpt_utils)

# Get train true values followed with predicted month/s
y_train_12m_list = y_train_12m['m_return_target(t+12)'].tolist()
y_test_12m_list = y_test_pred_12m.tolist()
#
df_to_evaluate_12m = mpt_utils.get_df_from_pred_list(df_pct, y_train_12m_list, y_test_12m_list)
df_to_evaluate_12m = df_to_evaluate_12m - 1
df_to_evaluate_12m

importlib.reload(mpt_utils)

weights_12m, mu_12m, S_12m, weights_all_12m = mpt_utils.portfolio_and_plot(df_to_evaluate_12m, df)

Expected annual return: 30.4%
Annual volatility: 28.4%
Sharpe Ratio: 1.00
-- Allocation --
{'AGYS': 1, '9107.T': 3, '2767.T': 2, 'SMCI': 1, '9101.T': 1, 'NVMI': 1, 'BBW': 1, 'DDS': 1, 'MOD': 1, '9104.T': 1, 'ANF': 1, '7936.T': 1, 'SKT': 1, '6430.T': 1, '9024.T': 1, '6146.T': 1, '8022.T': 1, '7433.T': 1, '4980.T': 1, 'AVGO': 1, 'DFIN': 1, 'ANET': 1, '7867.T': 1, 'UNM': 1, 'HOV': 1, 'INVP.L': 1, 'DECK': 1, 'DKS': 1, 'MEGP.L': 1, '6417.T': 2, 'LADR': 1, 'PAG': 1, '7740.T': 1, 'BELFB': 1, 'HRB': 1, 'CLMB': 1, 'CAL': 1, '7552.T': 1, '6857.T': 1, '7868.T': 9, 'NVDA': 1, '7994.T': 1, 'SFM': 1, '7762.T': 2, 'ONTO': 1, 'COHU': 1, '2760.T': 1, '9697.T': 1, '1911.T': 1, '8308.T': 3, '7734.T': 1, 'FINV': 3, 'CPRX': 1, '8923.T': 1, '3003.T': 1, '3608.T': 2, '9260.T': 1, 'BGC': 1, '5991.T': 1, '3107.T': 1, 'RELL': 1, '8601.T': 2, '7906.T': 2, '2395.T': 2, 'GME': 1, '7599.T': 3, 'STGW': 1, '7272.T': 1, '3289.T': 1, '2124.T': 2, '6814.T': 1, '3191.T': 1, '3048.T': 1, '6644.T': 1, '2715.T': 1, '8897.T'

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Overview table

In [52]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(weights_all_12m, mu_12m, S_12m, df_pct)

,Share Count,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
AGYS,1,-0.010114,35.36%,50.97%,-6.46%
9107.T,3,0.027045,19.36%,7.72%,-5.79%
2767.T,2,0.017159,31.63%,-34.83%,-30.65%
SMCI,1,0.035568,26.13%,61.38%,37.07%
9101.T,1,0.025000,24.50%,11.01%,5.06%
...,...,...,...,...,...
4293.T,3,0.011136,46.83%,40.60%,-0.90%
7241.T,1,0.011477,44.04%,27.49%,-27.01%
7944.T,4,0.018295,29.37%,-467.40%,876.44%
7955.T,1,0.008636,34.37%,-11.02%,-1.48%
